In [1]:
from benchmark import Benchmark
from quantum import H_from_qubo, Qobj
from MaxCut import MaxCut

from qutip.parallel import parallel_map, serial_map

from qaoa_and_tdvp import QAOA, QAOAResult, qaoa_tdvp_rhs, tdvp_optimize_qaoa, scipy_optimize, gradient_descent

import pickle

import numpy as np
from numpy.typing import NDArray
from typing import List, Tuple, Union, Optional, Callable, Any
import networkx as nx
import matplotlib.pyplot as plt
import plotly as py

from itertools import product

import pandas as pd
import swifter

import os

In [2]:
class bench_result(dict):
    def __init_subclass__(cls) -> None:
        return super().__init_subclass__()
    def __repr__(self) -> str:
        return f'{self.get("algorithm", np.NaN)}, C={self.get("value", np.NaN)}, delta={self.get("delta", np.NaN)}'

def bench_recursive(
    input: MaxCut | pd.DataFrame |pd.Series,
    p: int|None = None,
    optimizers: dict[str, bool] = {
        "tdvp": True,
        "scipy": True,
        "gradient_descent": True,
    },
    tollarance: float = 1e-2,
    auto_save: bool = False,
    path: str = None,
    print_msg:bool=True,
) -> pd.DataFrame:
    """Benchmark function for one maxcut instance and one p value

    Args:
        instance (MaxCut | pd.Series): The instance to benchmark or a pandas series having a field "instance" with the instances and a field "p" with the p value
        p (int): the qaoa depth
        optimizers (dict[str,bool]): dictionary of optimizers to use.
        Only keys "tdvp", "scipy" and "gradient_descent" are recognized

    Returns:
        dict: The result of the benchmark in a form that can be directly converted to a pandas dataframe
    """
    
    
    if isinstance(input, MaxCut):
        if print_msg:
            print(
                f"Running benchmark on instance with {input.graph.number_of_nodes()} with optimizers {(key for key,value in optimizers.items() if value)}"
            )
        qaoa = QAOA(input.qubo, p=p)
        delta_0 = tuple(1 for _ in range(2 * p))
        results: dict[str, QAOAResult] = dict()

        # get the results
        if optimizers.get("tdvp", False):
            print("optimizing with tdvp")
            tdvp_res = tdvp_optimize_qaoa(
                qaoa=qaoa,
                delta_0=delta_0,
                Delta=100,
                grad_tol=tollarance,
                int_mode="RK45",
                max_iter=100,
            )
            results["tdvp"] = tdvp_res

        if optimizers.get("scipy", False):
            print("optimizing with scipy")
            scipy_res = scipy_optimize(
                delta_0=delta_0, qaoa=qaoa, record_path=True, tol=tollarance
            )
            results["scipy"] = scipy_res

        if optimizers.get("gradient_descent", False):
            print("optimizing with gradient descent")
            gradient_descent_res = gradient_descent(
                delta_0=delta_0, qaoa=qaoa, tol=tollarance
            )
            results["gradient_descent"] = gradient_descent_res

        out = {
            algo:bench_result({
                "instance": input,
                # "qaoa": qaoa,   # produces unnecessary large files
                "p": p,
                "n": qaoa.n,
                "delta_0": delta_0,
                "tollarance": tollarance,
                "algorithm": algo,
                "res": res,
                "delta": res.parameters,
                "value": res.value,
                "path": res.parameter_path,
                "steps": res.num_steps,
                "num_fun_calls": res.num_fun_calls,
                "real duration": res.duration,
                "message": res.message,
            })
            for algo, res in results.items()
        }
        for k in out.keys():
            out[k].__repr__ = lambda: f"{k}: {out[k]['value']}"

        # return the results
        return pd.DataFrame(data=out, columns=['tdvp', 'scipy', 'gradient_descent'])

    elif isinstance(input, pd.Series):
        out = bench_recursive(
                    input['instance'],
                    p=input.name[0],
                    optimizers=optimizers,
                    tollarance=tollarance,
                    auto_save=auto_save,
                    path=path,
                    print_msg=False,
                )
        input['tdvp'],input['scipy'], input['gradient_descent'] = out['tdvp'], out['scipy'], out['gradient_descent']
        return input
    elif isinstance(input, pd.DataFrame):
        if print_msg:
            print(
                f"Running benchmark on {len(input)} instances with optimizers {(key for key,value in optimizers.items() if value)}"
            )
        return input.apply(lambda x:
            bench_recursive(x, optimizers=optimizers, tollarance=tollarance, auto_save=auto_save, path=path, print_msg=False),
            axis=1,
        )
    else:
        raise TypeError(
            f"instance must be either a MaxCut instance or a pandas series of MaxCut instances or a pandas Series but is {type(input)}"
        )
        
        


In [15]:
def bench_instance(
    input: MaxCut,
    p: int|None = None,
    optimizers: dict[str, bool] = {
        "tdvp": True,
        "scipy": True,
        "gradient_descent": True,
    },
    tollarance: float = 1e-2,
    auto_save: bool = False,
    path: str = None,
    print_msg:bool=True,
) -> pd.DataFrame:
    if print_msg:
            print(
                f"Running benchmark on instance with {input.graph.number_of_nodes()} with optimizers {(key for key,value in optimizers.items() if value)}"
            )
    qaoa = QAOA(input.qubo, p=p)
    delta_0 = tuple(1 for _ in range(2 * p))
    results: dict[str, QAOAResult] = dict()

    # get the results
    if optimizers.get("tdvp", False):
        print("optimizing with tdvp")
        tdvp_res = tdvp_optimize_qaoa(
            qaoa=qaoa,
            delta_0=delta_0,
            Delta=100,
            grad_tol=tollarance,
            int_mode="RK45",
            max_iter=100,
        )
        results["tdvp"] = tdvp_res

    if optimizers.get("scipy", False):
        print("optimizing with scipy")
        scipy_res = scipy_optimize(
            delta_0=delta_0, qaoa=qaoa, record_path=True, tol=tollarance
        )
        results["scipy"] = scipy_res

    if optimizers.get("gradient_descent", False):
        print("optimizing with gradient descent")
        gradient_descent_res = gradient_descent(
            delta_0=delta_0, qaoa=qaoa, tol=tollarance
        )
        results["gradient_descent"] = gradient_descent_res

    out = {
        algo:{
            "instance": input,
            # "qaoa": qaoa,   # produces unnecessary large files
            "p": p,
            "n": qaoa.n,
            "delta_0": delta_0,
            "tollarance": tollarance,
            "algorithm": algo,
            "res": res,
            "delta": res.parameters,
            "value": res.value,
            "path": res.parameter_path,
            "steps": res.num_steps,
            "num_fun_calls": res.num_fun_calls,
            "real duration": res.duration,
            "message": res.message,
        }
        for algo, res in results.items()
    }

    # return the results
    return pd.DataFrame(data=out, columns=['tdvp', 'scipy', 'gradient_descent'])
 
def bench_series(
    input: pd.Series,
    p: int|None = None,
    optimizers: dict[str, bool] = {
        "tdvp": True,
        "scipy": True,
        "gradient_descent": True,
    },
    tollarance: float = 1e-2,
    auto_save: bool = False,
    path: str = None,
    print_msg:bool=True,
) -> pd.DataFrame:
    out = bench_instance(
                input['instance'],
                p=input.name[0],
                optimizers=optimizers,
                tollarance=tollarance,
                auto_save=auto_save,
                path=path,
                print_msg=False,
            )
    input['tdvp'],input['scipy'], input['gradient_descent'] = out['tdvp'], out['scipy'], out['gradient_descent']
    return input
    
def bench_frame(
    input: pd.DataFrame,
    p: int|None = None,
    optimizers: dict[str, bool] = {
        "tdvp": True,
        "scipy": True,
        "gradient_descent": True,
    },
    tollarance: float = 1e-2,
    auto_save: bool = False,
    path: str = None,
    print_msg:bool=True,
) -> pd.DataFrame:
    if print_msg:
        print(
            f"Running benchmark on {len(input)} instances with optimizers {(key for key,value in optimizers.items() if value)}"
        )
    out = input.swifter.apply(lambda x:
        bench_series(x, optimizers=optimizers, tollarance=tollarance, auto_save=auto_save, path=path, print_msg=False),
        axis=1,
    )
    
    if path is not None:
        with open(path, "wb") as f:
            pickle.dump(out, f)
            
    return out


In [4]:
# n = 4
# p = 2
# with open(f"./instances/n{n}_instances.p","rb") as f:
#     instances = pickle.load(f)
# delta = tuple(1 for _ in range(2 * p))
# instances = dict(enumerate(instances))
# arrays=[
#     list(range(1,6)),
#     list(instances.keys()),
#     ['tdvp','scipy','gradient_descent']
# ]
# tuples = product(*arrays)
# index = pd.MultiIndex.from_tuples(tuples, names=["p","i","algorithm"])
# df = pd.DataFrame(index=index, columns=["instance", "result"])
# # df['instances'] = df.apply(lambda x: 0, axis=0)
# for (p,i, algo) in df.index:
#     df['instance'][(p,i,algo)] = instances[i]
# df

In [5]:
n = 4
p = 2
with open(f"./instances/n{n}_instances.p","rb") as f:
    instances = pickle.load(f)
delta = tuple(1 for _ in range(2 * p))
instances = dict(enumerate(instances))
arrays=[
    list(range(1,6)),
    list(instances.keys()),
]
tuples = product(*arrays)
index = pd.MultiIndex.from_tuples(tuples, names=["p","i"])
df = pd.DataFrame(index=index, columns=["instance", 'tdvp', 'scipy', 'gradient_descent'])
# df['instances'] = df.apply(lambda x: 0, axis=0)
for (p,i) in df.index:
    df['instance'][(p,i)] = instances[i]
df


instance tdvp scipy gradient_descent
p i                                                                      
1 0  <MaxCut.MaxCut object at 0x7f320b72be20>  NaN   NaN              NaN
  1  <MaxCut.MaxCut object at 0x7f320b77cc70>  NaN   NaN              NaN
  2  <MaxCut.MaxCut object at 0x7f31b7eeacb0>  NaN   NaN              NaN
  3  <MaxCut.MaxCut object at 0x7f31b7eebac0>  NaN   NaN              NaN
  4  <MaxCut.MaxCut object at 0x7f31b7ee81c0>  NaN   NaN              NaN
  5  <MaxCut.MaxCut object at 0x7f31b7eeb190>  NaN   NaN              NaN
2 0  <MaxCut.MaxCut object at 0x7f320b72be20>  NaN   NaN              NaN
  1  <MaxCut.MaxCut object at 0x7f320b77cc70>  NaN   NaN              NaN
  2  <MaxCut.MaxCut object at 0x7f31b7eeacb0>  NaN   NaN              NaN
  3  <MaxCut.MaxCut object at 0x7f31b7eebac0>  NaN   NaN              NaN
  4  <MaxCut.MaxCut object at 0x7f31b7ee81c0>  NaN   NaN              NaN
  5  <MaxCut.MaxCut object at 0x7f31b7eeb190>  NaN   NaN              NaN
3 0  <MaxCut.MaxCut object at 0x7f320b72be20>  NaN   NaN              NaN
  1  <MaxCut.MaxCut object at 0x7f320b77cc70>  NaN   NaN              NaN
  2  <MaxCut.MaxCut object at 0x7f31b7eeacb0>  NaN   NaN              NaN
  3  <MaxCut.MaxCut object at 0x7f31b7eebac0>  NaN   NaN              NaN
  4  <MaxCut.MaxCut object at 0x7f31b7ee81c0>  NaN   NaN              NaN
  5  <MaxCut.MaxCut object at 0x7f31b7eeb190>  NaN   NaN              NaN
4 0  <MaxCut.MaxCut object at 0x7f320b72be20>  NaN   NaN              NaN
  1  <MaxCut.MaxCut object at 0x7f320b77cc70>  NaN   NaN              NaN
  2  <MaxCut.MaxCut object at 0x7f31b7eeacb0>  NaN   NaN              NaN
  3  <MaxCut.MaxCut object at 0x7f31b7eebac0>  NaN   NaN              NaN
  4  <MaxCut.MaxCut object at 0x7f31b7ee81c0>  NaN   NaN              NaN
  5  <MaxCut.MaxCut object at 0x7f31b7eeb190>  NaN   NaN              NaN
5 0  <MaxCut.MaxCut object at 0x7f320b72be20>  NaN   NaN              NaN
  1  <MaxCut.MaxCut object at 0x7f320b77cc70>  NaN   NaN              NaN
  2  <MaxCut.MaxCut object at 0x7f31b7eeacb0>  NaN   NaN              NaN
  3  <MaxCut.MaxCut object at 0x7f31b7eebac0>  NaN   NaN              NaN
  4  <MaxCut.MaxCut object at 0x7f31b7ee81c0>  NaN   NaN              NaN
  5  <MaxCut.MaxCut object at 0x7f31b7eeb190>  NaN   NaN              NaN

In [6]:
bench_recursive(df['instance'].loc[1,0], 1, optimizers={'tdvp':False, 'scipy':True, 'gradient_descent':False}, tollarance=1e-1,)

Running benchmark on instance with 4 with optimizers <generator object bench_recursive.<locals>.<genexpr> at 0x7f31b7d50ac0>
optimizing with scipy
Done Scipy_optim



,tdvp,scipy,gradient_descent
algorithm,NaN,scipy,NaN
delta,NaN,"(0.7967443681366325, 0.356162038224893)",NaN
delta_0,NaN,"(1, 1)",NaN
instance,NaN,<MaxCut.MaxCut object at 0x7f320b72be20>,NaN
message,NaN,Optimization terminated successfully.,NaN
n,NaN,4,NaN
num_fun_calls,NaN,13,NaN
p,NaN,1,NaN
path,NaN,"[(1, 1), (2.0, 1.0), (1.0, 2.0), (0.3165890092...",NaN
real duration,NaN,0.118223,NaN


In [7]:
df.loc[1,0]

instance            <MaxCut.MaxCut object at 0x7f320b72be20>
tdvp                                                     NaN
scipy                                                    NaN
gradient_descent                                         NaN
Name: (1, 0), dtype: object

In [8]:
bench_recursive(df.loc[1,0], optimizers={'tdvp':False, 'scipy':True, 'gradient_descent':False}, tollarance=1e-1,)

optimizing with scipy
Done Scipy_optim



instance                     <MaxCut.MaxCut object at 0x7f320b72be20>
tdvp                algorithm        NaN
delta            NaN
delt...
scipy               algorithm                                     ...
gradient_descent    algorithm        NaN
delta            NaN
delt...
Name: (1, 0), dtype: object

In [9]:
bench_recursive(df.xs(1,level="p",drop_level=False), optimizers={'tdvp':False, 'scipy':True, 'gradient_descent':False}, tollarance=1e-1,)

Running benchmark on 6 instances with optimizers <generator object bench_recursive.<locals>.<genexpr> at 0x7f31b7d50dd0>
optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim



instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7f320b72be20>   
  1  <MaxCut.MaxCut object at 0x7f320b77cc70>   
  2  <MaxCut.MaxCut object at 0x7f31b7eeacb0>   
  3  <MaxCut.MaxCut object at 0x7f31b7eebac0>   
  4  <MaxCut.MaxCut object at 0x7f31b7ee81c0>   
  5  <MaxCut.MaxCut object at 0x7f31b7eeb190>   

                                                  tdvp  \
p i                                                      
1 0  algorithm        NaN
delta            NaN
delt...   
  1  algorithm        NaN
delta            NaN
delt...   
  2  algorithm        NaN
delta            NaN
delt...   
  3  algorithm        NaN
delta            NaN
delt...   
  4  algorithm        NaN
delta            NaN
delt...   
  5  algorithm        NaN
delta            NaN
delt...   

                                                 scipy  \
p i                                                      
1 0  algorithm                                     ...   
  1  algorithm                                     ...   
  2  algorithm                                     ...   
  3  algorithm                                     ...   
  4  algorithm                                     ...   
  5  algorithm                                     ...   

                                      gradient_descent  
p i                                                     
1 0  algorithm        NaN
delta            NaN
delt...  
  1  algorithm        NaN
delta            NaN
delt...  
  2  algorithm        NaN
delta            NaN
delt...  
  3  algorithm        NaN
delta            NaN
delt...  
  4  algorithm        NaN
delta            NaN
delt...  
  5  algorithm        NaN
delta            NaN
delt...

In [16]:
bench_frame(df.xs(1,level="p",drop_level=False), optimizers={'tdvp':False, 'scipy':True, 'gradient_descent':False}, tollarance=1e-1, path="./results/n4_p1.p")

Running benchmark on 6 instances with optimizers <generator object bench_frame.<locals>.<genexpr> at 0x7f31ae992d50>


Pandas Apply:   0%|          | 0/6 [00:00<?, ?it/s]

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim

optimizing with scipy
Done Scipy_optim



instance  \
p i                                             
1 0  <MaxCut.MaxCut object at 0x7f320b72be20>   
  1  <MaxCut.MaxCut object at 0x7f320b77cc70>   
  2  <MaxCut.MaxCut object at 0x7f31b7eeacb0>   
  3  <MaxCut.MaxCut object at 0x7f31b7eebac0>   
  4  <MaxCut.MaxCut object at 0x7f31b7ee81c0>   
  5  <MaxCut.MaxCut object at 0x7f31b7eeb190>   

                                                  tdvp  \
p i                                                      
1 0  algorithm        NaN
delta            NaN
delt...   
  1  algorithm        NaN
delta            NaN
delt...   
  2  algorithm        NaN
delta            NaN
delt...   
  3  algorithm        NaN
delta            NaN
delt...   
  4  algorithm        NaN
delta            NaN
delt...   
  5  algorithm        NaN
delta            NaN
delt...   

                                                 scipy  \
p i                                                      
1 0  algorithm                                     ...   
  1  algorithm                                     ...   
  2  algorithm                                     ...   
  3  algorithm                                     ...   
  4  algorithm                                     ...   
  5  algorithm                                     ...   

                                      gradient_descent  
p i                                                     
1 0  algorithm        NaN
delta            NaN
delt...  
  1  algorithm        NaN
delta            NaN
delt...  
  2  algorithm        NaN
delta            NaN
delt...  
  3  algorithm        NaN
delta            NaN
delt...  
  4  algorithm        NaN
delta            NaN
delt...  
  5  algorithm        NaN
delta            NaN
delt...

In [ ]:
df.xs(3, level=1, drop_level=False)

In [71]:
df.loc[[1]]

Empty DataFrame
Columns: []
Index: [(1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]

In [72]:
df.loc[1]

""
i
0
1
2
3
4
5


In [146]:
dummy_dict = {
    "tdvp": {
        "a_tdvp": 0,
        "b_tdvp": 1,
    },
    "scipy": {
        "a_scipy": 0,
        "b_scipy": 1,
    },
    "gradient_descent": {
        "a_grad": 0,
        "b_grad": 1,
    },
}


def dummy_fun(x):
    x["tdvp_result"], x["scipy_result"], x["gradient_descent_result"] = dummy_dict["tdvp"], dummy_dict["scipy"], dummy_dict["gradient_descent"]
    return x


In [183]:
# df['tdvp_result'],df['scipy_result'], df['gradient_descent_result']  =
df.apply(
    lambda x: x.name[0],
    axis=1)
# df

p  i
1  0    1
   1    1
   2    1
   3    1
   4    1
   5    1
2  0    2
   1    2
   2    2
   3    2
   4    2
   5    2
3  0    3
   1    3
   2    3
   3    3
   4    3
   5    3
4  0    4
   1    4
   2    4
   3    4
   4    4
   5    4
5  0    5
   1    5
   2    5
   3    5
   4    5
   5    5
dtype: int64

In [3]:
min_p = 1
max_p = 5

for n in range(4, 9):
    with open(f"./instances/n{n}_instances.p","rb") as f:    
        instances = pickle.load(f)
    bench = Benchmark()

    for num, i in enumerate(instances):
        print(f"instance {num}/{len(instances)}")
        for p in range(min_p, max_p+1):
            print(f"\t p = {p}/{max_p}")
            bench.run(
                i,
                delta_0=tuple(1 for _ in range(2 * p)),
                p=p,
                tdvp_range=100,
                tollarance=1e-2,
                tdvp_lineq_solver="RK45",
            )
            bench.save(f"RK45_n{n}_p-1-8_Delta-1_benchmarks.p")

instance 0/6
	 p = 1/5
Done Scipy_optim

donestep 52

(0.7825060145768947, 1.1303744630416939)
	 p = 2/5
Done Scipy_optim

donestep 245

(0.6986787335706295, 0.3069408770804383)


AssertionError: 